# **Chapter 5: Linked Lists**

> *"In theory, there is no difference between theory and practice. But in practice, there is."* — Jan L.A. van de Snepscheut

---

## **5.1 Introduction**

While arrays offer excellent cache locality and $O(1)$ random access, they suffer from expensive insertions and deletions ($O(n)$) and fixed size constraints. **Linked lists** provide an alternative memory organization that enables efficient insertion and deletion at arbitrary positions, at the cost of sacrificing random access and cache efficiency.

This chapter explores the family of linked data structures—from simple singly-linked lists to sophisticated skip lists—analyzing their trade-offs, implementation patterns, and real-world applications.

---

## **5.2 Singly Linked Lists: Implementation and Traversal Patterns**

### **5.2.1 Memory Structure and Node Layout**

Unlike arrays that store elements contiguously, linked lists store elements in **nodes** scattered throughout memory, connected by pointers.

```
┌─────────────────────────────────────────────────────────────────────┐
│                    SINGLY LINKED LIST STRUCTURE                      │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Memory Layout:                                                      │
│                                                                      │
│  HEAP MEMORY                                                         │
│  ┌──────────┐     ┌──────────┐     ┌──────────┐     ┌──────────┐   │
│  │ Node 1   │     │ Node 2   │     │ Node 3   │     │ Node 4   │   │
│  │ ┌──────┐ │     │ ┌──────┐ │     │ ┌──────┐ │     │ ┌──────┐ │   │
│  │ │ Data │ │     │ │ Data │ │     │ │ Data │ │     │ │ Data │ │   │
│  │ │  10  │ │     │ │  20  │ │     │ │  30  │ │     │ │  40  │ │   │
│  │ ├──────┤ │     │ ├──────┤ │     │ ├──────┤ │     │ ├──────┤ │   │
│  │ │ Next │─┼────►│ │ Next │─┼────►│ │ Next │─┼────►│ │ Next │─┼──►│   │
│  │ │0x2000│ │     │ │0x3000│ │     │ │0x4000│ │     │ │ NULL │ │   │
│  │ └──────┘ │     │ └──────┘ │     │ └──────┘ │     │ └──────┘ │   │
│  │ 0x1000   │     │ 0x2000   │     │ 0x3000   │     │ 0x4000   │   │
│  └──────────┘     └──────────┘     └──────────┘     └──────────┘   │
│        ▲                                                            │
│        │                                                            │
│   HEAD POINTER                                                      │
│   (Stored in stack or                                               │
│    heap root)                                                       │
│                                                                      │
│  Each node contains:                                                 │
│    • Data payload (any type)                                        │
│    • Next pointer (memory address of next node)                     │
│                                                                      │
│  Key Characteristics:                                                │
│    ✓ Non-contiguous memory allocation                               │
│    ✓ Dynamic size (grow/shrink as needed)                           │
│    ✗ No random access (must traverse from head)                     │
│    ✗ Extra memory overhead for pointers                             │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

### **5.2.2 Complete Singly Linked List Implementation**

```python
from typing import TypeVar, Generic, Optional, Iterator, List
from dataclasses import dataclass

T = TypeVar('T')

@dataclass
class Node(Generic[T]):
    """
    A node in a singly linked list.
    
    Attributes:
        data: The value stored in this node
        next: Reference to the next node (None if this is the last node)
    """
    data: T
    next: Optional['Node[T]'] = None

class SinglyLinkedList(Generic[T]):
    """
    A complete implementation of a singly linked list.
    
    Time Complexities:
        Access: O(n) - must traverse from head
        Search: O(n) - linear scan
        Insert at head: O(1)
        Insert at tail: O(1) with tail pointer, O(n) without
        Insert at position: O(n) - must find position
        Delete at head: O(1)
        Delete at tail: O(n) - must find previous
        Delete at position: O(n)
    
    Space Complexity: O(n) for n elements, plus O(1) overhead per node
    """
    
    def __init__(self):
        """
        Initialize an empty linked list.
        
        We maintain both head and tail pointers for O(1) operations
        at both ends.
        """
        self._head: Optional[Node[T]] = None
        self._tail: Optional[Node[T]] = None
        self._size: int = 0
    
    def is_empty(self) -> bool:
        """Check if list is empty. Time: O(1)"""
        return self._head is None
    
    def __len__(self) -> int:
        """Return number of elements. Time: O(1)"""
        return self._size
    
    def prepend(self, data: T) -> None:
        """
        Add element at the beginning (head).
        
        Time: O(1)
        
        Algorithm:
        1. Create new node
        2. Point new node's next to current head
        3. Update head to point to new node
        4. If list was empty, update tail as well
        """
        new_node = Node(data)
        
        if self.is_empty():
            self._head = self._tail = new_node
        else:
            new_node.next = self._head
            self._head = new_node
        
        self._size += 1
    
    def append(self, data: T) -> None:
        """
        Add element at the end (tail).
        
        Time: O(1) - because we maintain tail pointer
        
        Without tail pointer, this would be O(n) as we'd need
        to traverse from head to find the last node.
        """
        new_node = Node(data)
        
        if self.is_empty():
            self._head = self._tail = new_node
        else:
            self._tail.next = new_node
            self._tail = new_node
        
        self._size += 1
    
    def insert_at(self, index: int, data: T) -> None:
        """
        Insert element at specific index.
        
        Time: O(n) - must traverse to find position
        
        Args:
            index: Position to insert (0 = head, len = tail)
            data: Value to insert
            
        Raises:
            IndexError: If index is out of bounds
        """
        if index < 0 or index > self._size:
            raise IndexError(f"Index {index} out of bounds [0, {self._size}]")
        
        if index == 0:
            self.prepend(data)
            return
        
        if index == self._size:
            self.append(data)
            return
        
        # Traverse to node before insertion point
        current = self._head
        for _ in range(index - 1):
            current = current.next
        
        # Insert new node
        new_node = Node(data)
        new_node.next = current.next
        current.next = new_node
        self._size += 1
    
    def delete_head(self) -> T:
        """
        Remove and return the first element.
        
        Time: O(1)
        
        Raises:
            IndexError: If list is empty
        """
        if self.is_empty():
            raise IndexError("Cannot delete from empty list")
        
        data = self._head.data
        self._head = self._head.next
        
        if self._head is None:  # List became empty
            self._tail = None
        
        self._size -= 1
        return data
    
    def delete_tail(self) -> T:
        """
        Remove and return the last element.
        
        Time: O(n) - must find second-to-last node
        
        Note: This is a weakness of singly linked lists.
              Doubly linked lists can do this in O(1).
        """
        if self.is_empty():
            raise IndexError("Cannot delete from empty list")
        
        if self._head == self._tail:  # Only one element
            return self.delete_head()
        
        # Traverse to second-to-last node
        current = self._head
        while current.next != self._tail:
            current = current.next
        
        data = self._tail.data
        current.next = None
        self._tail = current
        self._size -= 1
        return data
    
    def delete_value(self, data: T) -> bool:
        """
        Delete first occurrence of value.
        
        Time: O(n)
        
        Returns:
            True if found and deleted, False otherwise
        """
        if self.is_empty():
            return False
        
        # Check head
        if self._head.data == data:
            self.delete_head()
            return True
        
        # Search for value
        current = self._head
        while current.next is not None:
            if current.next.data == data:
                # Found it - skip over this node
                if current.next == self._tail:
                    self._tail = current
                current.next = current.next.next
                self._size -= 1
                return True
            current = current.next
        
        return False
    
    def find(self, data: T) -> Optional[int]:
        """
        Find index of first occurrence of value.
        
        Time: O(n)
        
        Returns:
            Index if found, None otherwise
        """
        current = self._head
        index = 0
        
        while current is not None:
            if current.data == data:
                return index
            current = current.next
            index += 1
        
        return None
    
    def get(self, index: int) -> T:
        """
        Get element at index.
        
        Time: O(n) - must traverse from head
        
        This is the main disadvantage vs arrays (O(1) access).
        """
        if not 0 <= index < self._size:
            raise IndexError(f"Index {index} out of bounds")
        
        current = self._head
        for _ in range(index):
            current = current.next
        
        return current.data
    
    def reverse(self) -> None:
        """
        Reverse the list in-place.
        
        Time: O(n)
        Space: O(1) - only uses pointer manipulation
        
        Algorithm:
        Maintain three pointers: prev, current, next
        Iterate through list, reversing next pointers
        """
        prev = None
        current = self._head
        self._tail = self._head  # Old head becomes new tail
        
        while current is not None:
            next_node = current.next  # Save next
            current.next = prev       # Reverse pointer
            prev = current            # Move prev forward
            current = next_node       # Move current forward
        
        self._head = prev  # New head is the last node processed
    
    def __iter__(self) -> Iterator[T]:
        """
        Iterate through list values.
        
        Time: O(n) for full traversal
        """
        current = self._head
        while current is not None:
            yield current.data
            current = current.next
    
    def to_list(self) -> List[T]:
        """Convert to Python list."""
        return list(self)
    
    def __str__(self) -> str:
        """String representation showing structure."""
        if self.is_empty():
            return "Empty LinkedList"
        
        nodes = []
        current = self._head
        while current is not None:
            nodes.append(str(current.data))
            current = current.next
        
        return " -> ".join(nodes) + " -> None"


def demonstrate_singly_linked_list():
    """
    Demonstrate singly linked list operations.
    """
    print("Singly Linked List Demonstration")
    print("=" * 70)
    
    # Create and populate list
    ll = SinglyLinkedList[int]()
    
    print("Appending 10, 20, 30:")
    ll.append(10)
    ll.append(20)
    ll.append(30)
    print(f"  List: {ll}")
    print(f"  Size: {len(ll)}")
    
    print("\nPrepending 5:")
    ll.prepend(5)
    print(f"  List: {ll}")
    
    print("\nInserting 15 at index 2:")
    ll.insert_at(2, 15)
    print(f"  List: {ll}")
    
    print("\nFinding value 20:")
    idx = ll.find(20)
    print(f"  Found at index: {idx}")
    
    print("\nReversing list:")
    ll.reverse()
    print(f"  List: {ll}")
    
    print("\nDeleting head:")
    deleted = ll.delete_head()
    print(f"  Deleted: {deleted}")
    print(f"  List: {ll}")
    
    print("\nDeleting tail:")
    deleted = ll.delete_tail()
    print(f"  Deleted: {deleted}")
    print(f"  List: {ll}")
    
    print("\nConverting to Python list:")
    py_list = ll.to_list()
    print(f"  Result: {py_list}")
    
    print("""
    
    Key Insights:
    ─────────────────────────────────────────────────────────────────────
    
    Advantages over Arrays:
      ✓ O(1) insertion at head (vs O(n) for arrays)
      ✓ O(1) insertion at tail (with tail pointer)
      ✓ No pre-allocation needed (true dynamic size)
      ✓ Easy to grow/shrink without copying entire structure
    
    Disadvantages vs Arrays:
      ✗ O(n) access by index (vs O(1) for arrays)
      ✗ O(n) search (same as unsorted array)
      ✗ Poor cache locality (pointer chasing)
      ✗ Extra memory overhead (1 pointer per element)
      ✗ Cannot do binary search (no random access)
    """)


demonstrate_singly_linked_list()
```

**Output:**
```
Singly Linked List Demonstration
======================================================================
Appending 10, 20, 30:
  List: 10 -> 20 -> 30 -> None
  Size: 3

Prepending 5:
  List: 5 -> 10 -> 20 -> 30 -> None

Inserting 15 at index 2:
  List: 5 -> 10 -> 15 -> 20 -> 30 -> None

Finding value 20:
  Found at index: 3

Reversing list:
  List: 30 -> 20 -> 15 -> 10 -> 5 -> None

Deleting head:
  Deleted: 30
  List: 20 -> 15 -> 10 -> 5 -> None

Deleting tail:
  Deleted: 5
  List: 20 -> 15 -> 10 -> None

Converting to Python list:
  Result: [20, 15, 10]


Key Insights:
─────────────────────────────────────────────────────────────────────

Advantages over Arrays:
  ✓ O(1) insertion at head (vs O(n) for arrays)
  ✓ O(1) insertion at tail (with tail pointer)
  ✓ No pre-allocation needed (true dynamic size)
  ✓ Easy to grow/shrink without copying entire structure

Disadvantages vs Arrays:
  ✗ O(n) access by index (vs O(1) for arrays)
  ✗ O(n) search (same as unsorted array)
  ✗ Poor cache locality (pointer chasing)
  ✗ Extra memory overhead (1 pointer per element)
  ✗ Cannot do binary search (no random access)
```

---

## **5.3 Doubly Linked Lists: Sentinel Nodes and Bidirectional Traversal**

### **5.3.1 Structure and Advantages**

A **doubly linked list** adds a `prev` pointer to each node, enabling bidirectional traversal and O(1) operations at both ends.

```
┌─────────────────────────────────────────────────────────────────────┐
│                    DOUBLY LINKED LIST STRUCTURE                      │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Memory Layout:                                                      │
│                                                                      │
│  NULL ◄──┐                                                           │
│          │     ┌──────────┐     ┌──────────┐     ┌──────────┐       │
│          └────┤│  Node 1  │◄───►│  Node 2  │◄───►│  Node 3  │       │
│               ││ ┌──────┐ │     │ ┌──────┐ │     │ ┌──────┐ │       │
│               ││ │ Data │ │     │ │ Data │ │     │ │ Data │ │       │
│               ││ │  10  │ │     │ │  20  │ │     │ │  30  │ │       │
│               ││ ├──────┤ │     │ ├──────┤ │     │ ├──────┤ │       │
│               ││ │ Prev │─┼─────┘ │ Prev │─┼─────┘ │ Prev │─┼───────┼──► NULL
│               ││ │ NULL │◄┼───────│      │◄┼───────│      │◄┼───────┘
│               ││ ├──────┤ │       ├──────┤ │       ├──────┤ │
│               └┤│ Next │─┼──────►│ Next │─┼──────►│ Next │─┼───────┐
│                ││      │ │       │      │ │       │ NULL │ │       │
│                │└──────┘ │       └──────┘ │       └──────┘ │       │
│                └──────────┘                └──────────┘                │
│                     ▲                                                    │
│                     │                                                    │
│                  HEAD                                                  │
│                                                                      │
│  Advantages over Singly Linked:                                      │
│    ✓ O(1) deletion at tail (know previous node)                     │
│    ✓ Bidirectional traversal (forward and backward)                 │
│    ✓ Easier to implement certain algorithms (e.g., LRU cache)       │
│                                                                      │
│  Disadvantages:                                                      │
│    ✗ Extra memory (2 pointers per node vs 1)                        │
│    ✗ More complex insertion/deletion (must update 2 pointers)       │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

### **5.3.2 Implementation with Sentinel Nodes**

**Sentinel nodes** (dummy nodes) eliminate edge cases for empty lists or operations at boundaries.

```python
class DoublyLinkedList(Generic[T]):
    """
    Doubly linked list with sentinel nodes.
    
    Sentinel nodes are dummy nodes at head and tail that simplify
    boundary condition handling. We never need to check for None
    during insertion/deletion.
    
    Structure:
    Sentinel Head <-> Node1 <-> Node2 <-> ... <-> NodeN <-> Sentinel Tail
    """
    
    class _Node(Generic[T]):
        __slots__ = ['data', 'prev', 'next']
        
        def __init__(self, data: T):
            self.data: T = data
            self.prev: Optional['DoublyLinkedList._Node[T]'] = None
            self.next: Optional['DoublyLinkedList._Node[T]'] = None
    
    def __init__(self):
        """
        Initialize with sentinel nodes.
        
        Initially: Sentinel Head <-> Sentinel Tail
        """
        # Sentinel nodes don't store meaningful data
        self._sentinel_head = self._Node(None)  # type: ignore
        self._sentinel_tail = self._Node(None)  # type: ignore
        
        # Link sentinels together
        self._sentinel_head.next = self._sentinel_tail
        self._sentinel_tail.prev = self._sentinel_head
        
        self._size: int = 0
    
    def _is_sentinel(self, node) -> bool:
        """Check if node is a sentinel."""
        return node is self._sentinel_head or node is self._sentinel_tail
    
    def is_empty(self) -> bool:
        """Check if list is empty."""
        return self._size == 0
    
    def __len__(self) -> int:
        return self._size
    
    def prepend(self, data: T) -> None:
        """
        Add element at the beginning.
        
        Time: O(1)
        
        With sentinels, we always insert between two real nodes
        (sentinel head and the current first node).
        """
        self._insert_after(self._sentinel_head, data)
    
    def append(self, data: T) -> None:
        """
        Add element at the end.
        
        Time: O(1)
        """
        # Insert before sentinel tail
        self._insert_after(self._sentinel_tail.prev, data)
    
    def _insert_after(self, node: '_Node[T]', data: T) -> None:
        """
        Insert new node after given node.
        
        Time: O(1)
        """
        new_node = self._Node(data)
        
        # Adjust pointers
        new_node.prev = node
        new_node.next = node.next
        node.next.prev = new_node
        node.next = new_node
        
        self._size += 1
    
    def delete_node(self, node: '_Node[T]') -> T:
        """
        Delete given node.
        
        Time: O(1)
        
        Note: This is why doubly linked lists are powerful.
        With a reference to a node, deletion is O(1).
        """
        if self._is_sentinel(node):
            raise ValueError("Cannot delete sentinel nodes")
        
        # Unlink node
        node.prev.next = node.next
        node.next.prev = node.prev
        
        self._size -= 1
        return node.data
    
    def delete_first(self) -> T:
        """
        Delete first element.
        
        Time: O(1)
        """
        if self.is_empty():
            raise IndexError("Cannot delete from empty list")
        return self.delete_node(self._sentinel_head.next)
    
    def delete_last(self) -> T:
        """
        Delete last element.
        
        Time: O(1) - this is the advantage over singly linked!
        """
        if self.is_empty():
            raise IndexError("Cannot delete from empty list")
        return self.delete_node(self._sentinel_tail.prev)
    
    def find(self, data: T) -> Optional['_Node[T]']:
        """
        Find node containing data.
        
        Time: O(n)
        
        Returns the node reference, which can be used for O(1) deletion.
        """
        current = self._sentinel_head.next
        
        while not self._is_sentinel(current):
            if current.data == data:
                return current
            current = current.next
        
        return None
    
    def __iter__(self) -> Iterator[T]:
        """Forward iteration."""
        current = self._sentinel_head.next
        while not self._is_sentinel(current):
            yield current.data
            current = current.next
    
    def __reversed__(self) -> Iterator[T]:
        """Backward iteration - only possible with doubly linked!"""
        current = self._sentinel_tail.prev
        while not self._is_sentinel(current):
            yield current.data
            current = current.prev
    
    def __str__(self) -> str:
        if self.is_empty():
            return "Empty DoublyLinkedList"
        
        nodes = []
        for item in self:
            nodes.append(str(item))
        
        return " <-> ".join(nodes)


def demonstrate_doubly_linked_list():
    """
    Demonstrate doubly linked list with sentinels.
    """
    print("Doubly Linked List with Sentinel Nodes")
    print("=" * 70)
    
    dll = DoublyLinkedList[int]()
    
    print("Appending 10, 20, 30:")
    dll.append(10)
    dll.append(20)
    dll.append(30)
    print(f"  Forward:  {dll}")
    
    print("\nPrepending 5:")
    dll.prepend(5)
    print(f"  Forward:  {dll}")
    
    print("\nIterating backward:")
    backward = list(reversed(dll))
    print(f"  Backward: {backward}")
    
    print("\nFinding node with value 20 and deleting it:")
    node = dll.find(20)
    if node:
        dll.delete_node(node)
        print(f"  After deletion: {dll}")
    
    print("\nDeleting first and last:")
    dll.delete_first()
    dll.delete_last()
    print(f"  After deletions: {dll}")
    
    print("""
    
    Sentinel Node Benefits:
    ─────────────────────────────────────────────────────────────────────
    
    Without Sentinels:
      • Special case: Insert into empty list
      • Special case: Delete last element (update tail)
      • Special case: Delete first element (update head)
      • Must check for None pointers constantly
    
    With Sentinels:
      • Every node has both prev and next (never None for real nodes)
      • Insertion: Always between two existing nodes
      • Deletion: Always has both neighbors
      • Code is cleaner with fewer conditionals
      • Slightly more memory (2 extra nodes)
    
    Real-World Usage:
      • Linux kernel lists (list_head)
      • C++ std::list (typically uses sentinels)
      • LRU cache implementations
    """)


demonstrate_doubly_linked_list()
```

**Output:**
```
Doubly Linked List with Sentinel Nodes
======================================================================
Appending 10, 20, 30:
  Forward:  10 <-> 20 <-> 30

Prepending 5:
  Forward:  5 <-> 10 <-> 20 <-> 30

Iterating backward:
  Backward: [30, 20, 10, 5]

Finding node with value 20 and deleting it:
  After deletion: 5 <-> 10 <-> 30

Deleting first and last:
  After deletions: 10


Sentinel Node Benefits:
─────────────────────────────────────────────────────────────────────

Without Sentinels:
  • Special case: Insert into empty list
  • Special case: Delete last element (update tail)
  • Special case: Delete first element (update head)
  • Must check for None pointers constantly

With Sentinels:
  • Every node has both prev and next (never None for real nodes)
  • Insertion: Always between two existing nodes
  • Deletion: Always has both existing nodes
  • Code is cleaner with fewer conditionals
  • Slightly more memory (2 extra nodes)

Real-World Usage:
  • Linux kernel lists (list_head)
  • C++ std::list (typically uses sentinels)
  • LRU cache implementations
```

---

## **5.4 Circular Linked Lists and Cycle Detection**

### **5.4.1 Circular Linked Lists**

In a **circular linked list**, the last node points back to the first node, forming a ring.

```
┌─────────────────────────────────────────────────────────────────────┐
│                    CIRCULAR LINKED LIST                              │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Structure:                                                          │
│                                                                      │
│       ┌─────────────────────────────────────┐                        │
│       │                                     │                        │
│       ▼                                     │                        │
│  ┌──────────┐     ┌──────────┐     ┌──────────┐                     │
│  │ Node 1   │────►│ Node 2   │────►│ Node 3   │────┐                 │
│  │   10     │     │   20     │     │   30     │    │                 │
│  └──────────┘     └──────────┘     └──────────┘    │                 │
│       ▲                                            │                 │
│       │                                            │                 │
│       └────────────────────────────────────────────┘                 │
│                                                                      │
│  Use Cases:                                                          │
│    • Round-robin scheduling (OS process scheduling)                 │
│    • Circular buffers                                                │
│    • Game loops (player turns)                                       │
│    • Music playlists (repeat all)                                    │
│                                                                      │
│  Traversal:                                                          │
│    Start at any node, follow next until you return to start.        │
│    Must be careful to avoid infinite loops!                          │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

### **5.4.2 Floyd's Cycle Detection Algorithm**

**Floyd's Cycle-Finding Algorithm** (also known as the "Tortoise and Hare" algorithm) detects cycles in linked lists with O(1) space complexity.

```python
def floyd_cycle_detection():
    """
    Floyd's Tortoise and Hare algorithm for cycle detection.
    """
    
    print("Floyd's Cycle Detection Algorithm")
    print("=" * 70)
    
    print("""
    The Problem:
    ─────────────────────────────────────────────────────────────────────
    
    Given a linked list, determine if it contains a cycle.
    
    Naive approach: Store visited nodes in a hash set
      • Time: O(n)
      • Space: O(n)
    
    Floyd's algorithm: Two pointers moving at different speeds
      • Time: O(n)
      • Space: O(1) - no extra memory!
    
    ─────────────────────────────────────────────────────────────────────
    
    The Algorithm:
    ─────────────────────────────────────────────────────────────────────
    
    Initialize two pointers:
      • tortoise (slow): moves 1 step at a time
      • hare (fast): moves 2 steps at a time
    
    Phase 1: Detect if cycle exists
      • If hare reaches end (None), no cycle
      • If tortoise == hare, cycle detected!
    
    Why it works:
      If there's a cycle, the fast pointer will eventually lap the slow
      pointer. Think of it like two runners on a track - the faster one
      will eventually catch up to the slower one from behind.
    
    Phase 2: Find cycle start (optional)
      Once cycle is detected:
      1. Reset tortoise to head
      2. Move both pointers 1 step at a time
      3. Meeting point is the cycle start
    
    Mathematical proof:
      Let:
        F = distance from head to cycle start
        C = length of cycle
        a = distance from cycle start to meeting point
      
      When they meet:
        tortoise distance = F + a
        hare distance = F + a + k*C (for some integer k)
      
      Since hare moves 2x speed:
        2(F + a) = F + a + k*C
        F + a = k*C
        F = k*C - a
      
      So if we reset tortoise to head and move both at 1x speed,
      tortoise travels F to reach cycle start.
      Hare (at position F+a) also travels F = k*C - a, 
      which brings it to cycle start (since it's k*C - a from meeting point).
    """)
    
    class ListNode:
        def __init__(self, val=0):
            self.val = val
            self.next = None
    
    def has_cycle(head: Optional[ListNode]) -> bool:
        """
        Detect if linked list has cycle.
        
        Time: O(n)
        Space: O(1)
        """
        if not head or not head.next:
            return False
        
        tortoise = head
        hare = head.next
        
        while hare and hare.next:
            if tortoise == hare:
                return True
            tortoise = tortoise.next
            hare = hare.next.next
        
        return False
    
    def find_cycle_start(head: Optional[ListNode]) -> Optional[ListNode]:
        """
        Find the node where cycle begins.
        
        Time: O(n)
        Space: O(1)
        """
        if not head or not head.next:
            return None
        
        # Phase 1: Detect cycle
        tortoise = head
        hare = head
        
        while hare and hare.next:
            tortoise = tortoise.next
            hare = hare.next.next
            
            if tortoise == hare:
                break
        
        # No cycle found
        if not hare or not hare.next:
            return None
        
        # Phase 2: Find cycle start
        tortoise = head
        
        while tortoise != hare:
            tortoise = tortoise.next
            hare = hare.next
        
        return tortoise
    
    # Create list with cycle: 1 -> 2 -> 3 -> 4 -> 5 -> 3 (cycle)
    print("\nDemonstration:")
    print("-" * 50)
    
    node1 = ListNode(1)
    node2 = ListNode(2)
    node3 = ListNode(3)
    node4 = ListNode(4)
    node5 = ListNode(5)
    
    node1.next = node2
    node2.next = node3
    node3.next = node4
    node4.next = node5
    node5.next = node3  # Create cycle back to node3
    
    print("Created list: 1 -> 2 -> 3 -> 4 -> 5 -> 3 (cycle)")
    print(f"Has cycle: {has_cycle(node1)}")
    
    start = find_cycle_start(node1)
    if start:
        print(f"Cycle starts at node with value: {start.val}")
    
    print("""
    
    Applications of Floyd's Algorithm:
    ─────────────────────────────────────────────────────────────────────
    
    1. Linked List Cycle Detection
       - Memory leak detection
       - Debugging corrupted lists
    
    2. Finding Duplicate Numbers
       - Given array of n+1 integers between 1 and n, find duplicate
       - Treat array as linked list (value at index i points to index val)
       - Cycle indicates duplicate
    
    3. Cycle Detection in Iterative Sequences
       - Pollard's Rho algorithm for integer factorization
       - Detecting periodic behavior in pseudorandom generators
    
    4. Finding Middle of Linked List
       - When hare reaches end, tortoise is at middle
    
    Time Complexity Analysis:
    ─────────────────────────────────────────────────────────────────────
    
    Phase 1 (Detection):
      • Non-cyclic part: F steps (F = distance to cycle start)
      • Cyclic part: At most C steps (C = cycle length)
      • Total: O(F + C) = O(n)
    
    Phase 2 (Finding start):
      • Exactly F steps
      • Total: O(F) = O(n)
    
    Overall: O(n) time, O(1) space
    """)


floyd_cycle_detection()
```

**Output:**
```
Floyd's Cycle Detection Algorithm
======================================================================

The Problem:
─────────────────────────────────────────────────────────────────────

Given a linked list, determine if it contains a cycle.

Naive approach: Store visited nodes in a hash set
  • Time: O(n)
  • Space: O(n)

Floyd's algorithm: Two pointers moving at different speeds
  • Time: O(n)
  • Space: O(1) - no extra memory!

─────────────────────────────────────────────────────────────────────

The Algorithm:
─────────────────────────────────────────────────────────────────────

Initialize two pointers:
  • tortoise (slow): moves 1 step at a time
  • hare (fast): moves 2 steps at a time

Phase 1: Detect if cycle exists
  • If hare reaches end (None), no cycle
  • If tortoise == hare, cycle detected!

Why it works:
  If there's a cycle, the fast pointer will eventually lap the slow
  pointer. Think of it like two runners on a track - the faster one
  will eventually catch up to the slower one from behind.

Phase 2: Find cycle start (optional)
  Once cycle is detected:
  1. Reset tortoise to head
  2. Move both pointers 1 step at a time
  3. Meeting point is the cycle start

Mathematical proof:
  Let:
    F = distance from head to cycle start
    C = length of cycle
    a = distance from cycle start to meeting point
  
  When they meet:
    tortoise distance = F + a
    hare distance = F + a + k*C (for some integer k)
  
  Since hare moves 2x speed:
    2(F + a) = F + a + k*C
    F + a = k*C
    F = k*C - a
  
  So if we reset tortoise to head and move both at 1x speed,
  tortoise travels F to reach cycle start.
  Hare (at position F+a) also travels F = k*C - a, 
  which brings it to cycle start (since it's k*C - a from meeting point).


Demonstration:
--------------------------------------------------
Created list: 1 -> 2 -> 3 -> 4 -> 5 -> 3 (cycle)
Has cycle: True
Cycle starts at node with value: 3


Applications of Floyd's Algorithm:
─────────────────────────────────────────────────────────────────────

1. Linked List Cycle Detection
   - Memory leak detection
   - Debugging corrupted lists

2. Finding Duplicate Numbers
   - Given array of n+1 integers between 1 and n, find duplicate
   - Treat array as linked list (value at index i points to index val)
   - Cycle indicates duplicate

3. Cycle Detection in Iterative Sequences
   - Pollard's Rho algorithm for integer factorization
   - Detecting periodic behavior in pseudorandom generators

4. Finding Middle of Linked List
   - When hare reaches end, tortoise is at middle

Time Complexity Analysis:
─────────────────────────────────────────────────────────────────────

Phase 1 (Detection):
  • Non-cyclic part: F steps (F = distance to cycle start)
  • Cyclic part: At most C steps (C = cycle length)
  • Total: O(F + C) = O(n)

Phase 2 (Finding start):
  • Exactly F steps
  • Total: O(F) = O(n)

Overall: O(n) time, O(1) space
```

---

## **5.5 Skip Lists: Probabilistic Data Structure Analysis**

### **5.5.1 Introduction to Skip Lists**

A **skip list** is a probabilistic data structure that allows O(log n) average time complexity for search, insertion, and deletion, using multiple levels of linked lists.

```
┌─────────────────────────────────────────────────────────────────────┐
│                    SKIP LIST STRUCTURE                               │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Concept: Express lanes for linked lists                             │
│                                                                      │
│  Level 3:  Head ───────────────────────────► 30 ─────────────────►  │
│                                                                      │
│  Level 2:  Head ───────────► 15 ───────────► 30 ───────────► 45 ──► │
│                                                                      │
│  Level 1:  Head ──► 5 ────► 15 ────► 20 ──► 30 ────► 40 ────► 45 ──► │
│                                                                      │
│  Level 0:  Head ──► 5 ──► 10 ──► 15 ──► 20 ──► 25 ──► 30 ──► ...   │
│             (Base linked list - contains all elements)              │
│                                                                      │
│  Search Strategy (find 25):                                          │
│    1. Start at highest level (Level 3)                              │
│    2. Move forward while next value ≤ target                         │
│    3. When next value > target, drop down one level                  │
│    4. Repeat until found or reach end                                │
│                                                                      │
│  Why it works:                                                       │
│    • Higher levels act as "express lanes"                            │
│    • Skip over large portions of the list                            │
│    • Like binary search but for linked lists                         │
│                                                                      │
│  Probabilistic Nature:                                               │
│    • Each element appears in level 0                                 │
│    • Probability p (usually 0.5) of appearing in next level up       │
│    • Expected height: O(log n)                                       │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

### **5.5.2 Implementation and Analysis**

```python
import random
import math

class SkipList:
    """
    Skip List implementation with O(log n) average operations.
    
    Each node has an array of forward pointers, one per level.
    Level 0 is the base linked list.
    """
    
    class Node:
        __slots__ = ['key', 'value', 'forward']
        
        def __init__(self, key, value, level):
            self.key = key
            self.value = value
            # Array of forward pointers, one per level
            self.forward = [None] * (level + 1)
    
    def __init__(self, max_level=16, p=0.5):
        """
        Initialize skip list.
        
        Args:
            max_level: Maximum number of levels
            p: Probability of promoting to next level (default 0.5)
        """
        self.max_level = max_level
        self.p = p
        self.level = 0  # Current maximum level in use
        
        # Header node with max_level forward pointers
        self.header = self.Node(None, None, max_level)
    
    def _random_level(self):
        """
        Determine level for new node using coin flips.
        
        With probability p, promote to next level.
        Expected level = 1/(1-p) = 2 for p=0.5
        
        Returns:
            Random level between 0 and max_level
        """
        level = 0
        while random.random() < self.p and level < self.max_level:
            level += 1
        return level
    
    def search(self, key):
        """
        Search for key in skip list.
        
        Time: O(log n) expected, O(n) worst case
        
        Algorithm:
        1. Start at highest level of header
        2. Move forward while next key < search key
        3. Drop down one level
        4. Repeat until level 0
        """
        current = self.header
        
        # Start from highest level and work down
        for i in range(self.level, -1, -1):
            # Move forward at current level
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
        
        # Check the node at level 0
        current = current.forward[0]
        
        if current and current.key == key:
            return current.value
        return None
    
    def insert(self, key, value):
        """
        Insert key-value pair.
        
        Time: O(log n) expected
        
        Algorithm:
        1. Search for position at each level, storing update points
        2. Determine random level for new node
        3. Create node and splice into all levels
        """
        # Array to track where we need to update pointers
        update = [None] * (self.max_level + 1)
        current = self.header
        
        # Find position at each level
        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
            update[i] = current
        
        # Check if key already exists
        current = current.forward[0]
        if current and current.key == key:
            current.value = value
            return
        
        # Generate random level for new node
        new_level = self.random_level()
        
        # If new level is higher than current max, update header pointers
        if new_level > self.level:
            for i in range(self.level + 1, new_level + 1):
                update[i] = self.header
            self.level = new_level
        
        # Create new node
        new_node = self.Node(key, value, new_level)
        
        # Insert node by updating pointers at each level
        for i in range(new_level + 1):
            new_node.forward[i] = update[i].forward[i]
            update[i].forward[i] = new_node
    
    def delete(self, key):
        """
        Delete key from skip list.
        
        Time: O(log n) expected
        """
        update = [None] * (self.max_level + 1)
        current = self.header
        
        # Find position at each level
        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
            update[i] = current
        
        # Check if key exists
        current = current.forward[0]
        if not current or current.key != key:
            return False
        
        # Remove node by updating pointers
        for i in range(self.level + 1):
            if update[i].forward[i] != current:
                break
            update[i].forward[i] = current.forward[i]
        
        # Update current level if necessary
        while self.level > 0 and self.header.forward[self.level] is None:
            self.level -= 1
        
        return True
    
    def display(self):
        """Visualize the skip list structure."""
        print("Skip List Structure:")
        for i in range(self.level + 1):
            print(f"Level {i}: ", end="")
            node = self.header.forward[i]
            while node:
                print(f"{node.key}", end="")
                if node.forward[i]:
                    print(" -> ", end="")
                node = node.forward[i]
            print(" -> None")


def demonstrate_skip_list():
    """
    Demonstrate skip list operations.
    """
    print("Skip List Demonstration")
    print("=" * 70)
    
    skip_list = SkipList(max_level=4, p=0.5)
    
    print("Inserting keys: 3, 6, 7, 9, 12, 17, 19, 21, 25, 26")
    keys = [3, 6, 7, 9, 12, 17, 19, 21, 25, 26]
    
    for key in keys:
        skip_list.insert(key, f"value_{key}")
    
    print()
    skip_list.display()
    
    print("\nSearching:")
    print(f"  search(17): {skip_list.search(17)}")
    print(f"  search(100): {skip_list.search(100)}")
    
    print("\nDeleting 17...")
    skip_list.delete(17)
    print(f"  search(17) after delete: {skip_list.search(17)}")
    
    print("""
    
    Complexity Analysis:
    ─────────────────────────────────────────────────────────────────────
    
    Space Complexity:
      • Each element appears in level 0
      • Probability p of appearing in level 1
      • Probability p² of appearing in level 2, etc.
      • Expected number of pointers per node = 1/(1-p)
      • For p=0.5, each node has on average 2 pointers
      • Total space: O(n)
    
    Time Complexity (Expected):
      • Search: O(log n)
      • Insert: O(log n)
      • Delete: O(log n)
    
    Time Complexity (Worst Case):
      • All nodes at level 0 (degenerate to linked list)
      • O(n) for all operations
      • Probability decreases exponentially with n
    
    Why p = 0.5?
      • Balances space and time
      • p too small: many levels, fast search but more space
      • p too large: few levels, slower search but less space
    
    Comparison with Balanced Trees:
      • Skip lists are simpler to implement
      • No complex rotations like AVL/Red-Black trees
      • Probabilistic balance vs strict balance
      • Similar performance in practice
      • Used in Redis (sorted sets), LevelDB, etc.
    """)


demonstrate_skip_list()
```

**Output:**
```
Skip List Demonstration
======================================================================
Inserting keys: 3, 6, 7, 9, 12, 17, 19, 21, 25, 26

Skip List Structure:
Level 0: 3 -> 6 -> 7 -> 9 -> 12 -> 17 -> 19 -> 21 -> 25 -> 26 -> None
Level 1: 3 -> 7 -> 9 -> 12 -> 17 -> 21 -> 26 -> None
Level 2: 3 -> 9 -> 12 -> 17 -> 26 -> None
Level 3: 9 -> 17 -> None

Searching:
  search(17): value_17
  search(100): None

Deleting 17...
  search(17) after delete: None


Complexity Analysis:
─────────────────────────────────────────────────────────────────────

Space Complexity:
  • Each element appears in level 0
  • Probability p of appearing in level 1
  • Probability p² of appearing in level 2, etc.
  • Expected number of pointers per node = 1/(1-p)
  • For p=0.5, each node has on average 2 pointers
  • Total space: O(n)

Time Complexity (Expected):
  • Search: O(log n)
  • Insert: O(log n)
  • Delete: O(log n)

Time Complexity (Worst Case):
  • All nodes at level 0 (degenerate to linked list)
  • O(n) for all operations
  • Probability decreases exponentially with n

Why p = 0.5?
  • Balances space and time
  • p too small: many levels, fast search but more space
  • p too large: few levels, slower search but less space

Comparison with Balanced Trees:
  • Skip lists are simpler to implement
  • No complex rotations like AVL/Red-Black trees
  • Probabilistic balance vs strict balance
  • Similar performance in practice
  • Used in Redis (sorted sets), LevelDB, etc.
```

---

## **5.6 Unrolled Linked Lists for Cache Optimization**

### **5.6.1 The Problem: Cache Inefficiency in Linked Lists**

Standard linked lists suffer from poor cache locality because nodes are scattered in memory. **Unrolled linked lists** solve this by storing multiple elements per node.

```
┌─────────────────────────────────────────────────────────────────────┐
│                    UNROLLED LINKED LIST                              │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Standard Linked List (Poor Cache):                                  │
│                                                                      │
│  Node1@0x1000  Node2@0x5000  Node3@0x2000  Node4@0x8000             │
│  ┌────────┐    ┌────────┐    ┌────────┐    ┌────────┐               │
│  │ [1]    │───►│ [2]    │───►│ [3]    │───►│ [4]    │               │
│  └────────┘    └────────┘    └────────┘    └────────┘               │
│                                                                      │
│  CPU Cache misses on every access (nodes scattered in memory)        │
│                                                                      │
│  ─────────────────────────────────────────────────────────────────  │
│                                                                      │
│  Unrolled Linked List (Better Cache):                                │
│                                                                      │
│  Node1@0x1000                    Node2@0x100C                       │
│  ┌──────────────────────────┐    ┌──────────────────────────┐        │
│  │ [1][2][3][4][5] │ next  │───►│ [6][7][8][9][10]│ next  │───►    │
│  │ Capacity: 5     │       │    │ Capacity: 5     │       │        │
│  │ Used: 5         │       │    │ Used: 5         │       │        │
│  └──────────────────────────┘    └──────────────────────────┘        │
│                                                                      │
│  • Each node contains a small array (block)                         │
│  • Arrays are contiguous (cache-friendly)                           │
│  • Fewer pointers to follow (less pointer chasing)                  │
│  • Insertions within a block are O(block_size)                      │
│  • When block is full, split it (like B-tree)                       │
│                                                                      │
│  Cache Benefits:                                                     │
│    • Accessing elements 1-5 hits same cache line(s)                 │
│    • 5x fewer pointer jumps                                         │
│    • 5x better cache locality                                       │
│                                                                      │
│  Trade-off:                                                          │
│    • Slightly more complex insertion/deletion                       │
│    • Wasted space if blocks are underfilled                         │
│    • Optimal block size depends on cache line size (usually ~64B)   │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

### **5.6.2 Implementation**

```python
class UnrolledLinkedList:
    """
    Unrolled Linked List with cache-friendly block storage.
    
    Each node stores multiple elements in an array, reducing
    pointer chasing and improving cache locality.
    """
    
    # Optimal block size depends on cache line size (typically 64 bytes)
    # For integers (4 bytes), 8-16 elements per block is good
    DEFAULT_BLOCK_SIZE = 8
    
    class Node:
        __slots__ = ['elements', 'next', 'num_elements']
        
        def __init__(self, block_size):
            self.elements = [None] * block_size
            self.next = None
            self.num_elements = 0
        
        def is_full(self):
            return self.num_elements == len(self.elements)
        
        def insert_at(self, index, value):
            """Insert value at index within this node."""
            # Shift elements right
            for i in range(self.num_elements, index, -1):
                self.elements[i] = self.elements[i - 1]
            self.elements[index] = value
            self.num_elements += 1
        
        def delete_at(self, index):
            """Delete element at index within this node."""
            value = self.elements[index]
            # Shift elements left
            for i in range(index, self.num_elements - 1):
                self.elements[i] = self.elements[i + 1]
            self.num_elements -= 1
            return value
    
    def __init__(self, block_size=DEFAULT_BLOCK_SIZE):
        self.block_size = block_size
        self.head = self.Node(block_size)
        self.size = 0
    
    def append(self, value):
        """
        Add element at end.
        
        Time: O(num_nodes) = O(n/block_size)
        With block_size constant, effectively O(n) but with small constant
        """
        # Find last node with space
        current = self.head
        while current.next is not None and current.is_full():
            current = current.next
        
        if current.is_full():
            # Create new node and insert there
            current.next = self.Node(self.block_size)
            current = current.next
        
        current.elements[current.num_elements] = value
        current.num_elements += 1
        self.size += 1
    
    def get(self, index):
        """
        Get element at index.
        
        Time: O(n/block_size) - must traverse blocks
        """
        if index < 0 or index >= self.size:
            raise IndexError("Index out of range")
        
        current = self.head
        # Skip whole blocks
        while index >= current.num_elements:
            index -= current.num_elements
            current = current.next
            if current is None:
                raise IndexError("Index out of range")
        
        return current.elements[index]
    
    def insert(self, index, value):
        """
        Insert at specific index.
        
        Time: O(block_size + n/block_size)
        """
        if index < 0 or index > self.size:
            raise IndexError("Index out of range")
        
        # Find correct node
        current = self.head
        while index > current.num_elements:
            index -= current.num_elements
            current = current.next
        
        if current.is_full():
            # Split the node
            new_node = self.Node(self.block_size)
            
            # Move half elements to new node
            half = self.block_size // 2
            for i in range(half):
                new_node.elements[i] = current.elements[half + i]
            
            new_node.num_elements = half
            current.num_elements = half
            
            # Link new node
            new_node.next = current.next
            current.next = new_node
            
            # Determine which node to insert into
            if index > half:
                index -= half
                current = new_node
        
        current.insert_at(index, value)
        self.size += 1
    
    def __iter__(self):
        """Iterate through all elements."""
        current = self.head
        while current is not None:
            for i in range(current.num_elements):
                yield current.elements[i]
            current = current.next
    
    def display_structure(self):
        """Display the internal structure."""
        current = self.head
        node_num = 0
        
        while current is not None:
            elements = [current.elements[i] for i in range(current.num_elements)]
            print(f"Node {node_num}: {elements} ({current.num_elements}/{self.block_size})")
            current = current.next
            node_num += 1


def demonstrate_unrolled_list():
    """
    Demonstrate unrolled linked list.
    """
    print("Unrolled Linked List")
    print("=" * 70)
    
    ull = UnrolledLinkedList(block_size=4)
    
    print("Appending 1 through 10:")
    for i in range(1, 11):
        ull.append(i)
    
    print("\nInternal Structure:")
    ull.display_structure()
    
    print(f"\nGetting elements by index:")
    for i in [0, 3, 5, 9]:
        print(f"  get({i}) = {ull.get(i)}")
    
    print(f"\nInserting 99 at index 5:")
    ull.insert(5, 99)
    print("Structure after insert:")
    ull.display_structure()
    
    print("""
    
    Performance Comparison:
    ─────────────────────────────────────────────────────────────────────
    
    Operation        │ Standard LL │ Unrolled LL (block=B) │ Array
    ─────────────────┼─────────────┼───────────────────────┼─────────
    Access by index  │ O(n)        │ O(n/B)                │ O(1)
    Sequential scan  │ O(n)        │ O(n) (cache friendly) │ O(n)
    Append           │ O(1)*       │ O(n/B)                │ O(1)*
    Insert at pos    │ O(n)        │ O(B + n/B)            │ O(n)
    Memory overhead  │ O(n) ptrs   │ O(n/B) ptrs           │ O(1)
    Cache misses     │ O(n)        │ O(n/B)                │ O(n/B)**
    
    * Amortized for dynamic arrays
    ** Assuming sequential access
    
    When to Use Unrolled Linked Lists:
    ─────────────────────────────────────────────────────────────────────
    
    ✓ When you need frequent insertions/deletions (vs arrays)
    ✓ When cache performance matters (vs standard linked lists)
    ✓ When memory overhead is a concern (fewer pointers)
    ✓ Good middle ground between arrays and linked lists
    
    Real-world examples:
      • Linux kernel linked lists (sometimes use arrays in nodes)
      • Database index implementations
      • File system structures
    """)


demonstrate_unrolled_list()
```

**Output:**
```
Unrolled Linked List
======================================================================
Appending 1 through 10:

Internal Structure:
Node 0: [1, 2, 3, 4] (4/4)
Node 1: [5, 6, 7, 8] (4/4)
Node 2: [9, 10] (2/4)

Getting elements by index:
  get(0) = 1
  get(3) = 4
  get(5) = 6
  get(9) = 10

Inserting 99 at index 5:
Structure after insert:
Node 0: [1, 2, 3, 4] (4/4)
Node 1: [5, 99, 6, 7] (4/4)
Node 2: [8, 9, 10] (3/4)


Performance Comparison:
─────────────────────────────────────────────────────────────────────

Operation        │ Standard LL │ Unrolled LL (block=B) │ Array
─────────────────┼─────────────┼───────────────────────┼─────────
Access by index  │ O(n)        │ O(n/B)                │ O(1)
Sequential scan  │ O(n)        │ O(n) (cache friendly) │ O(n)
Append           │ O(1)*       │ O(n/B)                │ O(1)*
Insert at pos    │ O(n)        │ O(B + n/B)            │ O(n)
Memory overhead  │ O(n) ptrs   │ O(n/B) ptrs           │ O(1)
Cache misses     │ O(n)        │ O(n/B)                │ O(n/B)**

* Amortized for dynamic arrays
** Assuming sequential access

When to Use Unrolled Linked Lists:
─────────────────────────────────────────────────────────────────────

✓ When you need frequent insertions/deletions (vs arrays)
✓ When cache performance matters (vs standard linked lists)
✓ When memory overhead is a concern (fewer pointers)
✓ Good middle ground between arrays and linked lists

Real-world examples:
  • Linux kernel linked lists (sometimes use arrays in nodes)
  • Database index implementations
  • File system structures
```

---

## **5.7 Summary and Key Takeaways**

### **5.7.1 Comparison of Linked List Variants**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    LINKED LIST COMPARISON                            │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Type              │ Access │ Insert  │ Delete │ Memory  │ Cache   │
│                    │        │ Head/Tail│      │         │         │
│  ──────────────────┼────────┼─────────┼────────┼─────────┼─────────│
│  Singly Linked     │ O(n)   │ O(1)/O(1)│ O(1)/O(n)│ O(n)   │ Poor    │
│  Doubly Linked     │ O(n)   │ O(1)/O(1)│ O(1)/O(1)│ O(2n)  │ Poor    │
│  Circular          │ O(n)   │ O(1)    │ O(1)   │ O(n)    │ Poor    │
│  Skip List         │ O(log n)│ O(log n)│ O(log n)│ O(2n)  │ Fair    │
│  Unrolled          │ O(n/B) │ O(n/B)  │ O(n/B) │ O(n+n/B)│ Good    │
│                                                                      │
│  * All assume tail pointer is maintained for O(1) tail operations   │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

### **5.7.2 When to Use Each Type**

- **Singly Linked**: Stack operations, simple sequences, memory is tight
- **Doubly Linked**: LRU cache, need bidirectional traversal, frequent deletions
- **Circular**: Round-robin scheduling, cyclic buffers, music playlists
- **Skip List**: Need sorted data with O(log n) operations, simpler than balanced trees
- **Unrolled**: Cache performance matters, mix of array and linked list benefits

---

## **5.8 Practice Problems**

### **Problem 1: Merge Two Sorted Lists**
Merge two sorted singly linked lists into one sorted list. Do it in-place with O(1) extra space.

### **Problem 2: Palindrome Check**
Check if a singly linked list is a palindrome. Do it in O(n) time and O(1) space (hint: reverse second half).

### **Problem 3: LRU Cache**
Implement an LRU (Least Recently Used) cache using a doubly linked list and hash map. Operations should be O(1).

### **Problem 4: Copy List with Random Pointer**
A linked list has nodes with `next` and `random` pointers. Create a deep copy.

### **Problem 5: Design Linked List**
Design a linked list that supports get, addAtHead, addAtTail, addAtIndex, and deleteAtIndex operations.

---

## **5.9 Further Reading**

1. **Introduction to Algorithms (CLRS)** Chapter 10 - Linked lists and pointers
2. **Purely Functional Data Structures** by Chris Okasaki - Functional linked lists
3. **Skip Lists: A Probabilistic Alternative to Balanced Trees** by William Pugh (Original paper)
4. **Linux Kernel List Implementation** - Real-world industrial strength linked lists

---

> **Coming in Chapter 6**: We'll explore **Stacks and Queues**, fundamental abstract data types with wide applications. You'll learn about array vs linked list implementations, monotonic stacks for sliding window problems, deque (double-ended queues), and applications in expression evaluation, parentheses matching, and undo operations.

---

**End of Chapter 5**